In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from utils import *
#from utils_cpdir import *
from utils_mvcp import *
import os

In [2]:
d = 120
d_v = 40
K = 6
N = 3000
L = 20
train_prop = .3
N_train = int(N * train_prop)
scale = 3
corr = 0
var = .2

alpha = .1

set_size = defaultdict(list)
cov = defaultdict(list)
cov_cal = defaultdict(list)

seed = 0
#s1, s2, s3, y_train, y_test = simulate_s(N, d, .5, .5, scale, K, N_train, seed)

for i in range(10):
    if i % 5 == 0:
        print(i)
        
    seed = seed + i
    U_list = gen_u(K, seed=seed)
    
    for k in range(1, K+1):
        #print(k)
        p = (np.arange(k)+1) ** 1.5
        p = p / p.sum()
        p
        score_list, s_pre_fused, y_train, y_test, top_views = simulate_s(N, d, L, scale, corr, var, k, N_train, seed, adapt=True)

        quant_smx_list = []
        rest_smx_list = []
        cal_smx_list = []
        val_smx_list = []
            
        for j in range(k+1):
            quant_smx, rest_smx, quant_labels, rest_labels, idx_quant = split_cal_test(score_list[j], y_test, cal_prop=0.2, seed=seed)
            cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(rest_smx, rest_labels, cal_prop=0.8, seed=seed)

            quant_smx_list.append(quant_smx)
            rest_smx_list.append(rest_smx)
            cal_smx_list.append(cal_smx)
            val_smx_list.append(val_smx)

        n = cal_labels.shape[0]

        single_view_ss = []
        single_view_cov = []
        for j in range(k):

            s_u = rest_smx_list[j]
            cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
            n = cal_smx.shape[0]
            (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
            single_view_ss.append(prediction_sets.sum(1).mean())
            single_view_cov.append(empirical_coverage)

        if k > 1:
            prediction_sets, empirical_coverage = mvcp_2(quant_smx_list[:-1],
                                                             quant_labels,
                                                             cal_smx_list[:-1], 
                                                             cal_labels, 
                                                             val_smx_list[:-1], 
                                                             alpha, 
                                                             val_labels, 
                                                             U=U_list[k-2])

            set_size['mvcp'+str(k)].append(
                    prediction_sets.sum(1).mean()
            )
            cov['mvcp'+str(k)].append(
                    empirical_coverage
            )
        else:
            
            s_u = rest_smx_list[-1]

            cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
            n = cal_smx.shape[0]
            (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
            
            set_size['mvcp'+str(k)].append(
                    prediction_sets.sum(1).mean()
            )
            cov['mvcp'+str(k)].append(
                    empirical_coverage
            )
                
        s_u = rest_smx_list[-1]

        cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
        n = cal_smx.shape[0]
        (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
        set_size['full'+str(k)].append(prediction_sets.sum(1).mean())
        cov['full'+str(k)].append(empirical_coverage)
        
            #s_u = rest_smx_list[j]
        idx_best = np.argmin(single_view_ss)
        set_size['view'+str(k)].append(single_view_ss[idx_best])
        cov['view'+str(j+1)].append(single_view_cov[idx_best])

        s_u = np.sum(rest_smx_list[:-1], 0)

        cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
        n = cal_smx.shape[0]
        (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
        set_size['sum'+str(k)].append(prediction_sets.sum(1).mean())
        cov['sum'+str(k)].append(empirical_coverage)

        s_u = np.power(rest_smx_list[:-1],2).sum(0)

        cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
        n = cal_smx.shape[0]
        (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
        set_size['norm'+str(k)].append(prediction_sets.sum(1).mean())
        cov['norm'+str(k)].append(empirical_coverage)
        
        #s_u = 1/(np.array([1+(np.argmax(score_list[:-1], 2)[:,~idx_quant]==l).mean(0) for l in range(L)]).T)  
        #s_u = 1-np.product(score_list[:-1], 0)[~idx_quant,] ** (1/6)
        #s_u = np.power(rest_smx_list[:-1],2).sum(0)
        #s_u = 1-np.max(score_list[:-1], 0)[~idx_quant,]
        #s_u = np.min(rest_smx_list[:-1], 0)
        s_u = s_pre_fused[~idx_quant,:]
        
        cal_smx, val_smx, cal_labels, val_labels, idx = split_cal_test(s_u, rest_labels, cal_prop=0.8, seed=seed)
        n = cal_smx.shape[0]
        (prediction_sets, empirical_coverage), qhat = conform_prediction(cal_smx, val_smx, cal_labels, n, alpha, val_labels, cond=0)
        set_size['pre_fused'+str(k)].append(prediction_sets.sum(1).mean())
        cov['pre_fused'+str(k)].append(empirical_coverage)
        

0
5


In [11]:
df_res = pd.DataFrame(pd.DataFrame(set_size).mean()).reset_index()
df_res['cp'] = [s[:-1] for s in df_res['index']]
df_res['M'] = [s[-1] for s in df_res['index']]
del df_res['index']
df_res = df_res.pivot(index='M', columns='cp', values=0).loc[:,['view','pre_fused','sum','norm', 'mvcp']]
df_res

cp,view,pre_fused,sum,norm,mvcp
M,,,,,
1,6.036310,6.036310,6.036310,6.036310,6.036310
2,10.837202,8.109524,8.727976,8.720238,7.583929
3,12.356845,8.494643,9.049405,8.832738,7.683333
4,13.320536,8.753274,9.043452,8.413690,8.074107
5,13.897321,9.007440,8.913393,8.447321,8.371131
6,14.275000,9.263393,9.041964,8.709226,8.708631


In [4]:
print(df_res.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
)) 

\begin{tabular}{rrrrr}
\toprule
 view &  pre\_fused &  sum &  norm &  mvcp \\
 6.04 &       6.04 & 6.04 &  6.04 &  6.04 \\
\midrule
10.84 &       8.11 & 8.73 &  8.72 &  7.58 \\
12.36 &       8.49 & 9.05 &  8.83 &  7.68 \\
13.32 &       8.75 & 9.04 &  8.41 &  8.07 \\
13.90 &       9.01 & 8.91 &  8.45 &  8.37 \\
14.28 &       9.26 & 9.04 &  8.71 &  8.71 \\
\bottomrule
\end{tabular}



/var/folders/9g/bj6gc6z17_1fyb7lx57xf04m0000gs/T/ipykernel_72716/2737531582.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res.to_latex(index=False,


In [12]:
df_res_std = pd.DataFrame(pd.DataFrame(set_size).std()).reset_index()
df_res_std['cp'] = [s[:-1] for s in df_res_std['index']]
df_res_std['M'] = [s[-1] for s in df_res_std['index']]
del df_res_std['index']
df_res_std = df_res_std.pivot(index='M', columns='cp', values=0).loc[:,['view','pre_fused','sum','norm', 'mvcp']]
df_res_std

cp,view,pre_fused,sum,norm,mvcp
M,,,,,
1,0.420534,0.420534,0.420534,0.420534,0.420534
2,0.337252,0.557598,0.415499,0.421479,0.420751
3,0.300163,0.435591,0.315823,0.353717,0.424288
4,0.300004,0.514955,0.466093,0.504901,0.503555
5,0.193914,0.608646,0.630007,0.586314,0.575063
6,0.266406,0.559473,0.678224,0.706984,0.746006


In [8]:
print(df_res_std.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
)) 

\begin{tabular}{lrrrrr}
\toprule
cp &  view &  pre\_fused &  sum &  norm &  mvcp \\
M &       &            &      &       &       \\
\midrule
1 &  0.42 &       0.42 & 0.42 &  0.42 &  0.42 \\
2 &  0.34 &       0.56 & 0.42 &  0.42 &  0.42 \\
3 &  0.30 &       0.44 & 0.32 &  0.35 &  0.42 \\
4 &  0.30 &       0.51 & 0.47 &  0.50 &  0.50 \\
5 &  0.19 &       0.61 & 0.63 &  0.59 &  0.58 \\
6 &  0.27 &       0.56 & 0.68 &  0.71 &  0.75 \\
\bottomrule
\end{tabular}



/var/folders/9g/bj6gc6z17_1fyb7lx57xf04m0000gs/T/ipykernel_72716/3078087418.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res_std.to_latex(index=True,


In [9]:
df_res = pd.DataFrame(pd.DataFrame(cov).mean()).reset_index()
df_res['cp'] = [s[:-1] for s in df_res['index']]
df_res['M'] = [s[-1] for s in df_res['index']]
del df_res['index']
df_res = df_res.pivot(index='M', columns='cp', values=0).loc[:,['view','pre_fused','sum','norm', 'mvcp']]
df_res

cp,view,pre_fused,sum,norm,mvcp
M,,,,,
1,0.910119,0.910119,0.910119,0.910119,0.910119
2,0.892262,0.908333,0.908036,0.907738,0.901488
3,0.891369,0.910417,0.909226,0.909524,0.902679
4,0.888690,0.909524,0.904167,0.904167,0.908333
5,0.881250,0.903571,0.901488,0.898810,0.900298
6,0.889881,0.905655,0.895536,0.897917,0.900893


In [10]:
print(df_res.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
)) 

\begin{tabular}{lrrrrr}
\toprule
cp &  view &  pre\_fused &  sum &  norm &  mvcp \\
M &       &            &      &       &       \\
\midrule
1 &  0.91 &       0.91 & 0.91 &  0.91 &  0.91 \\
2 &  0.89 &       0.91 & 0.91 &  0.91 &  0.90 \\
3 &  0.89 &       0.91 & 0.91 &  0.91 &  0.90 \\
4 &  0.89 &       0.91 & 0.90 &  0.90 &  0.91 \\
5 &  0.88 &       0.90 & 0.90 &  0.90 &  0.90 \\
6 &  0.89 &       0.91 & 0.90 &  0.90 &  0.90 \\
\bottomrule
\end{tabular}



/var/folders/9g/bj6gc6z17_1fyb7lx57xf04m0000gs/T/ipykernel_72716/361595659.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res.to_latex(index=True,
